# Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q git+https://github.com/miladmozafari/SpykeTorch.git

  Preparing metadata (setup.py) ... done


In [ ]:

import numpy as np
import pandas as pd
import imageio
import os
import ast
import sys

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
from torch.nn.parameter import Parameter
import torchvision
import numpy as np
from SpykeTorch import snn
from SpykeTorch import functional as sf
from SpykeTorch import visualization as vis
from SpykeTorch import utils
from torchvision import transforms

dir = '/content/drive/MyDrive/Thesis/'
cifar_dir = '/content/drive/MyDrive/Thesis/Data/extracted'

# Open

In [ ]:
import SpykeTorch.utils as utils
import SpykeTorch.functional as sf
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
import random
import numpy as np


def time_dim(input):
    return input.unsqueeze(0)



transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
     time_dim,
     sf.pointwise_inhibition,
     utils.Intensity2Latency(number_of_spike_bins = 15, to_spike = True)])

# kernels = [	utils.GaborKernel(window_size = 3, orientation = 45+22.5),
#             utils.GaborKernel(3, 90+22.5),
#             utils.GaborKernel(3, 135+22.5),
#             utils.GaborKernel(3, 180+22.5)]
# filter = utils.Filter(kernels, use_abs = True)

# def time_dim(input):
#     return input.unsqueeze(0)

# transform = transforms.Compose(
#     [transforms.Grayscale(),
#     transforms.ToTensor(),
#     time_dim,
#     filter,
#     sf.pointwise_inhibition,
#     utils.Intensity2Latency(number_of_spike_bins = 15, to_spike = True)])



# class Transform_Caltech:
#     def __init__(self,
#                     pooling_size,
#                     pooling_stride,
#                     lateral_inhibition = None,
#                     timesteps = 15,
#                     feature_wise_inhibition=True):
        
#         self.transform = transforms.Compose((
#             transforms.Grayscale(),
#             transforms.ToTensor()))
        
#         self.kernels = [utils.GaborKernel(5, 45+22.5),
#                         utils.GaborKernel(5, 90+22.5),
#                         utils.GaborKernel(5, 135+22.5),
#                         utils.GaborKernel(5, 180+22.5)]
        
#         self.filter = utils.Filter(self.kernels, use_abs = True)
#         self.pooling_size = pooling_size
#         self.pooling_stride = pooling_stride
#         self.lateral_inhibition = lateral_inhibition
#         self.in2lan = utils.Intensity2Latency(timesteps)
#         self.feature_wise_inhibition = feature_wise_inhibition
#     def __call__(self, image):
#         image = self.transform(image)
#         image.unsqueeze_(0)
#         image = self.filter(image)
#         image = sf.pooling(image, self.pooling_size, self.pooling_stride, padding=self.pooling_size//2)
#         if self.lateral_inhibition is not None:
#             image = self.lateral_inhibition(image)
#         temporal_image = self.in2lan(image)
#         temporal_image = sf.pointwise_inhibition(temporal_image)
#         return temporal_image.sign().byte()

# lateral_inhibition = utils.LateralIntencityInhibition([0.15, 0.12, 0.1, 0.07, 0.05])
# inp = Transform_Caltech(7, 6, lateral_inhibition)

In [ ]:
CIFAR_train = utils.CacheDataset(torchvision.datasets.CIFAR10(root=cifar_dir, train=True, download=True, transform = transform))
CIFAR_test = utils.CacheDataset(torchvision.datasets.CIFAR10(root=cifar_dir, train=False, download=True, transform = transform))
CIFAR_loader = DataLoader(CIFAR_train, batch_size=1000, shuffle=False)
CIFAR_testLoader = DataLoader(CIFAR_test, batch_size=len(CIFAR_test), shuffle=False)

images, labels = next(CIFAR_loader.__iter__())  

images[0].shape

Files already downloaded and verified
Files already downloaded and verified


torch.Size([15, 3, 32, 32])

# Model

In [ ]:
class CSNN(nn.Module):
    def __init__(self):
        super(CSNN, self).__init__()

        self.n_layers = 4

        # Pooling Layer
        self.pool = snn.Pooling(kernel_size = 3, stride = 1).cuda()

        # Conv Layers
        self.conv1 = snn.Convolution(in_channels=3, out_channels=64, kernel_size=4).cuda()
        self.conv2 = snn.Convolution(in_channels=64, out_channels=128, kernel_size=3).cuda()
        self.conv3 = snn.Convolution(in_channels=128, out_channels=256, kernel_size=3).cuda()
        self.conv4 = snn.Convolution(in_channels=256, out_channels=200, kernel_size=11).cuda()

        # STDP Functions
        self.stdp1 = snn.STDP(conv_layer = self.conv1, learning_rate = (0.1, -0.005)).cuda()  #0.1, -0.005
        self.stdp2 = snn.STDP(conv_layer = self.conv2, learning_rate = (0.07, -0.005)).cuda() #0.09, -0.005
        self.stdp3 = snn.STDP(conv_layer = self.conv3, learning_rate = (0.05, -0.005)).cuda()  #0.08 -0.005
        self.stdp4 = snn.STDP(self.conv4, (0.5, -0.05), False, 0.2).cuda() #(0.007, -0.01)
        self.anti_stdp = snn.STDP(self.conv4, (-0.0002, 0.075), False, 0.2).cuda() # (-0.002, 0.0001)

        # Spike Counts
        self.spk_cnt1 = 0
        self.spk_cnt2 = 0
        self.spk_cnt3 = 0

        # Max learning rate
        self.max_ap = Parameter(torch.Tensor([0.3]))

        # Feature to classes map
        self.feature2class = []
        for i in range(10):
            self.feature2class.extend([i]*20)

    def forward(self, x, t, layer):
        if self.training:
            # x = sf.pad(x.float(), (2,2,2,2))
            if layer >= 1:
                x = self.pool(x)
                p = self.conv1(x)
                o, p = sf.fire(p, 10, return_thresholded_potentials=True)
                if layer == 1:
                    self.spk_cnt1 += 1
                    if self.spk_cnt1 >= 500:
                        self.spk_cnt1 = 0
                        ap = torch.tensor(self.stdp1.learning_rate[0][0].item(), device=self.stdp1.learning_rate[0][0].device) * 20
                        ap = torch.min(ap, self.max_ap)
                        an = ap * -0.75
                        # self.stdp1.update_all_learning_rate(ap.item(), an.item())
                    p = sf.pointwise_inhibition(p)
                    o = p.sign()
                    winners = sf.get_k_winners(p, kwta=5, inhibition_radius=0, spikes=o)

            if layer >= 2: 
                # x = sf.pad(sf.pooling(o, 2, 2), (1,1,1,1))
                x = o
                p = self.conv2(o)
                o, p = sf.fire(p, 12, return_thresholded_potentials=True)
                if layer == 2:
                    self.spk_cnt2 += 1
                    if self.spk_cnt2 >= 500:
                        self.spk_cnt2 = 0
                        ap = torch.tensor(self.stdp2.learning_rate[0][0].item(), device=self.stdp2.learning_rate[0][0].device) * 2
                        ap = torch.min(ap, self.max_ap)
                        an = ap * -0.75
                        # self.stdp2.update_all_learning_rate(ap.item(), an.item())
                    p = sf.pointwise_inhibition(p)
                    o = p.sign()
                    winners = sf.get_k_winners(p, kwta=3, inhibition_radius=0, spikes=o)
            if layer >= 3:
                # x = sf.pad(sf.pooling(o, 2, 2), (2,2,2,2))
                # x = self.pool(o)
                x = o
                p = self.conv3(x)
                o, p = sf.fire(p, 5, return_thresholded_potentials=True)
                if layer == 3:
                    self.spk_cnt3 += 1
                    if self.spk_cnt3 >= 500:
                        self.spk_cnt3 = 0
                        ap = torch.tensor(self.stdp3.learning_rate[0][0].item(), device=self.stdp3.learning_rate[0][0].device) * 2
                        ap = torch.min(ap, self.max_ap)
                        an = ap * -0.75
                        # self.stdp3.update_all_learning_rate(ap.item(), an.item())
                    p = sf.pointwise_inhibition(p)
                    o = p.sign()
                    winners = sf.get_k_winners(p, kwta=2, inhibition_radius=0, spikes=o)
                    
            if layer == 4:
                x = o
                p = self.conv4(x)
                o = sf.fire(p)
                winners = sf.get_k_winners(p, kwta=1, inhibition_radius=0, spikes=o)
                if len(winners) != 0:
                    if self.feature2class[winners[0][0]] == t:
                        self.stdp(layer, x, p, o, winners)
                        return winners
                    elif layer == self.n_layers:
                        self.anti_stdp(x, p, o, winners)
                        return winners
                return winners

            # print(x.shape, o.shape, p.shape, len(winners))
            self.stdp(layer, x, p, o, winners)
            return winners

        else:
            x = self.pool(x.cuda())
            # x = sf.pad(x.float(), (2,2,2,2), 0)
            p = self.conv1(x)
            o, p = sf.fire(p, 10, return_thresholded_potentials=True)
            x = o
            # x = sf.pad(sf.pooling(o, 2, 2), (1,1,1,1))
            p = self.conv2(x)
            o, p = sf.fire(p, 5, return_thresholded_potentials=True)
            # x = self.pool(o)
            # x = sf.pad(sf.pooling(o, 2, 2), (2,2,2,2))
            x = o
            p = self.conv3(x)
            o, p = sf.fire(p, 2, return_thresholded_potentials=True)
            p = self.conv4(o)
            o = sf.fire(p)
            winners = sf.get_k_winners(p, kwta=1, inhibition_radius=0, spikes=o)
            return winners

    def reset(self):
        self.conv.reset_weight    

    def stdp(self, layer, x, p, o, winners):
        if layer == 1:
            self.stdp1(x, p, o, winners)
        elif layer == 2:
            self.stdp2(x, p, o, winners)
        elif layer == 3:
            self.stdp3(x, p, o, winners)
        else:
            self.stdp4(x, p, o, winners)


    def update_learning_rates(self, stdp_ap, stdp_an, anti_stdp_ap, anti_stdp_an):
        self.stdp4.update_all_learning_rate(stdp_ap, stdp_an)
        self.anti_stdp.update_all_learning_rate(anti_stdp_an, anti_stdp_ap)

      
def train(net, train_loader, layer, epoch=20):
    logs = []
    net.train()

    for iter in range(epoch):
        print('\rIteration:', iter, end="")
        for data,targets in train_loader:
            log = {'silent':0, 'total':0}
            for x,t in zip(data, targets):
                winners = net(x.cuda(), t.cuda(), layer)
                log['total'] += 1
                if len(winners) == 0:
                    log['silent'] += 1
        if (iter % 5 == 1 or iter == epoch - 1):
            print(log)
            logs.append(log)
    return logs
    print()
    print("Unsupervised Learning is Done.")


def train_rl(net, train_loader, epoch=20):
    logs = []
    net.train()
    # initial adaptive learning rates
    apr = net.stdp4.learning_rate[0][0].item()
    anr = net.stdp4.learning_rate[0][1].item()
    app = net.anti_stdp.learning_rate[0][1].item()
    anp = net.anti_stdp.learning_rate[0][0].item()

    adaptive_min = 0
    adaptive_int = 1
    apr_adapt = ((1.0 - 1.0 / 5) * adaptive_int + adaptive_min) * apr
    anr_adapt = ((1.0 - 1.0 / 5) * adaptive_int + adaptive_min) * anr
    app_adapt = ((1.0 / 5) * adaptive_int + adaptive_min) * app
    anp_adapt = ((1.0 / 5) * adaptive_int + adaptive_min) * anp

    best_test = 1.0

    for iter in range(epoch):
        
        print('\rIteration:', iter, end="")
        for data,targets in train_loader:
            log = {'silent':0, 'error':0,'total':0}
            for x,t in zip(data, targets):
                log['total'] += 1
                winners = net(x.cuda(), t.cuda(), 4)
                if len(winners) != 0:
                    if net.feature2class[winners[0][0]] != t:
                        log['error'] += 1
                else:
                    log['silent'] += 1 

                correct = log['total']-log['error']-log['silent']

                apr_adapt = apr * (log['error'] * adaptive_int + adaptive_min)
                anr_adapt = anr * (log['error'] * adaptive_int + adaptive_min)
                app_adapt = app * (correct * adaptive_int + adaptive_min)
                anp_adapt = anp * (correct * adaptive_int + adaptive_min)
                net.update_learning_rates(apr_adapt, anr_adapt, app_adapt, anp_adapt)

            err = test(net, 4, CIFAR_testLoader)
            if err < best_test:
                best_test = err
                print('NEW BEST: ' + str(best_test))
                torch.save(net.state_dict(), l4_dir)
            if err < 0.5:
                break
                
        if (iter % 5 == 1 or iter == epoch - 1):
            print(best_test)
            print('epoch ' + str(epoch))
            print(log)

    print()
    print("Reinforcement Learning is Done.")

def test(net, layer, test_loader):
    net.eval()
    error = 0
    silent = 0
    total = 0
    for data,targets in test_loader:
        for x,t in zip(data, targets):
            winners = net(x.cuda(), t.cuda(), layer)
            total += 1
            if len(winners) != 0:
                if net.feature2class[winners[0][0]] != t:
                    error += 1
            else:
                silent += 1  
    print("         Error:", error/total)
    print("Silent Samples:", silent/total)
    return error/total
 

## Training

In [ ]:
csnn = CSNN().cuda()

force_train = [0,0,0,0]

# Training The First Layer'
l1_dir = '/content/drive/MyDrive/Thesis/checkpoints/saved_l1.net'
print("Training the first layer")
if force_train[0]:
    train(csnn, CIFAR_loader, 1, 3)
    torch.save(csnn.state_dict(), l1_dir)
elif os.path.isfile(l1_dir):
    csnn.load_state_dict(torch.load(l1_dir))




Training the first layer


In [ ]:
# Training The Second Layer
l2_dir = '/content/drive/MyDrive/Thesis/checkpoints/saved_l2.net'
print("Training the second layer")
if force_train[1]:
    train(csnn, CIFAR_loader, 2, 5)
    torch.save(csnn.state_dict(), l2_dir)
elif os.path.isfile(l2_dir):
    csnn.load_state_dict(torch.load(l2_dir))

csnn = CSNN().cuda()

# Training The Third Layer
l3_dir = '/content/drive/MyDrive/Thesis/checkpoints/saved_l3.net'
print("Training the third layer")
if force_train[2]:
    train(csnn, CIFAR_loader, 3, 5)
    torch.save(csnn.state_dict(), l3_dir)
elif os.path.isfile(l3_dir):
    csnn.load_state_dict(torch.load(l3_dir))



Training the second layer
Training the third layer


In [ ]:


# Training The Fourth Layer
l4_dir = '/content/drive/MyDrive/Thesis/checkpoints/saved_l4.net'
print("Training the fourth layer")
if force_train[3]:
    train_rl(csnn, CIFAR_loader, 15)
elif os.path.isfile(l4_dir):
    csnn.load_state_dict(torch.load(l4_dir))

Training the fourth layer


In [ ]:
test(csnn, 4, CIFAR_testLoader) #0.35485

         Error: 0.9
Silent Samples: 0.0


0.9